In [1]:
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModel
import torch
import os 
cert = r"Zscaler Root CA.crt"
os.environ["REQUESTS_CA_BUNDLE"] = cert
model = AutoModel.from_pretrained("google/siglip-base-patch16-256-multilingual")
processor = AutoProcessor.from_pretrained("google/siglip-base-patch16-256-multilingual")

config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

C:\Users\F4L5GPL\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\F4L5GPL\.cache\huggingface\hub\models--google--siglip-base-patch16-256-multilingual. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

In [20]:
from PIL import Image

# url = "giay-ra-vien.jpg"
image = Image.open("hoa_don.png")

# texts = ["a photo of 2 cats", "a photo of 2 dogs"]
texts = ["bức ảnh là 1 tờ giấy", "bức ảnh là giấy ra viện" , "ảnh là hóa đơn gía trị gia tăng"]
inputs = processor(text=texts, images=image, padding="max_length", return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
print("Shape: ", outputs.keys())
logits_per_image = outputs.logits_per_image
probs = torch.sigmoid(logits_per_image) # these are the probabilities
print(f"{probs[0][0]:.1%} bức ảnh là '{texts[0]}'")
print(f"{probs[0][1]:.1%} bức ảnh là '{texts[1]}'")
print(f"{probs[0][2]:.1%} bức ảnh là '{texts[2]}'")

Shape:  odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])
1.2% bức ảnh là 'bức ảnh là 1 tờ giấy'
27.2% bức ảnh là 'bức ảnh là giấy ra viện'
98.6% bức ảnh là 'ảnh là hóa đơn gía trị gia tăng'


In [35]:
outputs['text_model_output']['pooler_output']

tensor([[ 0.5682, -0.1897, -0.9573,  ..., -0.1999,  0.2125, -0.1812],
        [ 0.4934,  0.0832, -0.8253,  ...,  0.0955, -0.0119, -0.1409],
        [ 0.1257,  0.4013, -0.8600,  ...,  0.4563,  0.3708,  0.3020]])

In [32]:
outputs['text_embeds']

tensor([[ 0.0250, -0.0084, -0.0422,  ..., -0.0088,  0.0094, -0.0080],
        [ 0.0223,  0.0038, -0.0373,  ...,  0.0043, -0.0005, -0.0064],
        [ 0.0057,  0.0183, -0.0392,  ...,  0.0208,  0.0169,  0.0138]])

In [36]:
outputs['text_model_output']['pooler_output'] / outputs['text_model_output']['pooler_output'].norm(p=2, dim=-1, keepdim=True)

tensor([[ 0.0250, -0.0084, -0.0422,  ..., -0.0088,  0.0094, -0.0080],
        [ 0.0223,  0.0038, -0.0373,  ...,  0.0043, -0.0005, -0.0064],
        [ 0.0057,  0.0183, -0.0392,  ...,  0.0208,  0.0169,  0.0138]])

In [27]:
outputs['image_embeds'].shape, outputs['text_embeds'].shape, outputs['text_model_output'][0].size(), outputs['vision_model_output'][0].size()

(torch.Size([1, 768]),
 torch.Size([3, 768]),
 torch.Size([3, 64, 768]),
 torch.Size([1, 256, 768]))

In [5]:
import shutil

import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
response = requests.get(url, stream=True)
with open('img.png', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)